# Individual vs Team COVID Improvisation

In [12]:
#importing packages
import pandas as pd
from bs4 import BeautifulSoup
import requests
import string
import sys

In [3]:
#import csv
articles = pd.read_csv("compiled_articles.csv")

In [4]:
articles.head()

,Unnamed: 0,Unnamed: 1,Title,Datetime,Source,URL
0,newsriv,0,10 more coronavirus patients die in Black Coun...,2020-05-17T13:58:06.998+0000,Express & Star,https://www.expressandstar.com/news/health/cor...
1,newsriv,1,Coronavirus fight could delay NHS staff apprai...,2020-06-02T17:32:23.477+0000,Shropshire Star,https://www.shropshirestar.com/news/health/cor...
2,newsriv,2,Covid-19 coronavirus: Top intensive care docto...,2020-03-23T20:46:36.671+0000,New Zealand Herald,https://www.nzherald.co.nz/nz/news/article.cfm...
3,newsriv,3,COVID-19 Call Center is the Place for Grays Ha...,2020-03-24T19:16:47.913+0000,graysharbortalk.com/,http://www.graysharbortalk.com/2020/03/24/covi...
4,newsriv,4,GP referrals drop across Black Country and Sta...,2020-05-27T05:08:59.719+0000,Express & Star,https://www.expressandstar.com/news/health/cor...


#### Subsetting for testing code

In [5]:
art = articles.iloc[250:300,:]
art.head()

,Unnamed: 0,Unnamed: 1,Title,Datetime,Source,URL
250,newsriv,250,How Kaggle Data Scientists Help with Coronavir...,2020-06-15T15:30:39.483+0000,Google,https://cloud.google.com/blog/products/ai-mach...
251,newsriv,251,EULAR launches COVID-19 RMD reporting database...,2020-03-31T18:26:06.658+0000,medicalxpress.com/,https://medicalxpress.com/news/2020-03-eular-c...
252,newsriv,252,5 paramedic and healthcare workers jobs in,2020-05-29T07:34:55.647+0000,Emergency-Live,https://www.emergency-live.com/of-interest/top...
253,newsriv,253,"With AI, Africa’s health systems can take on C...",2020-04-22T16:29:23.229+0000,iafrikan.com/,https://www.iafrikan.com/2020/04/22/artificial...
254,newsriv,254,SA Lockdown: SANParks ready to reopen,2020-05-25T12:58:33.768+0000,Lee YongHoon,https://www.enca.com/news/sa-lockdown-sanparks...


In [8]:
#looking at columns
articles.head(3)

,Unnamed: 0,Unnamed: 1,Title,Datetime,Source,URL
0,newsriv,0,10 more coronavirus patients die in Black Coun...,2020-05-17T13:58:06.998+0000,Express & Star,https://www.expressandstar.com/news/health/cor...
1,newsriv,1,Coronavirus fight could delay NHS staff apprai...,2020-06-02T17:32:23.477+0000,Shropshire Star,https://www.shropshirestar.com/news/health/cor...
2,newsriv,2,Covid-19 coronavirus: Top intensive care docto...,2020-03-23T20:46:36.671+0000,New Zealand Herald,https://www.nzherald.co.nz/nz/news/article.cfm...


### Collecting HTML from each article into a list

In [9]:
info = {}
for x in range(len(art)):
    url = art.iloc[x,5]
    r = requests.get(url)
    html_doc= r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    if soup.find_all('p'):
        y =""
        for x in soup.find_all('p'):
            y += x.get_text()
        info[soup.title.text] = y
    elif soup.find_all('section'):
        y = ""
        for x in soup.find_all('section'): #section class = 'articlebody'
            y += x.get_text()
        info[soup.title.text] = y
    
    

- The code above is the one that breaks

- The code below is a work in progress (it downloads 91/2794 articles) 7/11/2020

In [ ]:
info = {}
for x in range(len(articles)):
    try:
        url = articles.iloc[x,5]
        r = requests.get(url)
        html_doc= r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        #print(soup.prettify())
        if soup.find_all('p'):
            y =""
            for x in soup.find_all('p'):
                y += x.get_text()
            info[soup.title.text] = y
        elif soup.find_all('section'):
            y = ""
            for x in soup.find_all('section'): #section class = 'articlebody'
                y += x.get_text()
            info[soup.title.text] = y
            
    except:
        e = sys.exc_info()[0]
        print(e)
        print(x)
      

In [8]:
len(info)

42

### Tokenizing and Team Word Counts

In [7]:
#clean the text
counter = {}

for a,b in info.items():
    lower = b.lower()
    no_punc = lower.translate(str.maketrans('','',string.punctuation))
    token = no_punc.split()
    
    #word count
    together = 0
    doctors = 0
    nurses = 0
    team = 0 
    group = 0
    collaboration = 0
    teamwork = 0
    teamwork2 = 0
    
    #keyword list
    keywords = [together,doctors,nurses,team, group, collaboration, teamwork, teamwork2]
    
    #counting words in tokenized text
    for x in token:
        if x == "together":
            together += 1
        elif x == 'doctors':
            doctors += 1
        elif x == 'nurses':
            nurses += 1
        elif x == 'team':
            team += 1
        elif x =='group':
            group += 1
        elif x== 'collaboration':
            collaboration +=1
        elif x == 'teamwork':
            teamwork += 1
        elif x == 'team-work':
            teamwork2 += 1
    
    #counting up the number of keywords
    count = sum(keywords)
    
    counter[a] = count
    

In [8]:
#sort the articles based off of word count
sort_articles = sorted(counter.items(), key=lambda x: x[1], reverse=True)

for i in sort_articles:
    print(i[0],i[1])

{'10 more coronavirus patients die in Black Country, Staffordshire and Birmingham | Express & Star': 0,
 'Coronavirus fight could delay NHS staff appraisals | Shropshire Star': 0,
 ' Covid-19 coronavirus: Top intensive care doctor admits he is scared - NZ Herald': 0,
 'COVID-19 Call Center is the Place for Grays Harbor Residents to Ask Questions - GraysHarborTalk': 0,
 'GP referrals drop across Black Country and Staffordshire | Express & Star': 0,
 'Access Denied': 0,
 'Spain’s mass COVID-19 deaths tipped to surpass\xa0Italy | Daily Telegraph': 0,
 'International Medical Corps and MediSys Health Network Hospitals Partner to Respond to COVID-19 | PressReleasePoint': 0,
 'W.H.O declares Europe the new COVID-19\xa0epicentre | Daily Telegraph': 0,
 'W.H.O declares Europe the new COVID-19\xa0epicentre | Herald Sun': 0,
 'DVIDS - News - US Army Reserve FEST members assist USACE’s COVID19 response projects for Navajo Nation': 0,
 '\nZensors Offers AI Platform to Assist Cities & Businesses wit